<a href="https://colab.research.google.com/github/RyanTokManMokMTM/NLP_Training4/blob/main/Words2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip wiki_zh_2019
#unzip the zipfile

In [ ]:
from os import listdir
from os.path import isfile, isdir, join
import json
ALL_DATAS = []
filesList = {
    0:"AA",
    1:"AB",
    2:"AC",
    4:"AD",
    5:"AE",
    6:"AF",
    7:"AG",
    8:"AH",    
    9:"AI",
    10:"AJ",
    11:"AK",
    12:"AL",
    13:"AM"
}

for i in filesList:
  folderPath = "./wiki_zh/%s/" % filesList[i]
  files = listdir(folderPath)
  for i in files:
    curFilePath = folderPath + "%s"%i 
    print(curFilePath)
    with open(curFilePath) as f:
      for j in f.readlines():
        str2Json = json.loads(j)
        text = "".join(str2Json["text"].strip().split())
        ALL_DATAS.append(text)
      f.close()


In [ ]:
!pip install opencc-python-reimplemented

In [47]:
#convert text 
from opencc import OpenCC
#convert to zh-hk
ccConvertor = OpenCC("s2hk")
transChineseList = []
for i in ALL_DATAS:
  transChineseList.append(ccConvertor.convert(i))


In [55]:
#store convertd data to csv
import pandas as pd
datas = pd.DataFrame(transChineseList)
datas.to_csv("alldatas.csv",encoding="utf-8")

# cut words

In [56]:
!pip install jieba

In [58]:
#load stopwords list
import codecs 
fp = codecs.open("./stopwords.dat","r",encoding="utf-8")
contents = fp.read()
fp.close()

In [ ]:
import jieba
import jieba.analyse
cutWordsListDatas = []
x = 0;
for i in transChineseList:
  cutWordsListDatas.append(" ".join([word for word in jieba.cut(i,cut_all=False) if word not in contents]))
  if(x%5000 == 0):
    print(x)
  x+=1


In [94]:
cutWordsTxt = "cutWordsList.txt"
with open(cutWordsTxt,"w") as fw:
  for data in cutWordsListDatas[:50]:
    fw.write(data)
    fw.write("\n")

cutWordsListDatas[1]

'約克大學 加拿大 約克大學 法語 Universit é York 加拿大 安大略省 多倫多市 一家 著名 綜合性 大學 約克大學 工商管理 法律 美術 設計 電影 傳理 人文 學科 科學 心理 學等 專業 而聞名 工商管理 專業 倫敦 經濟 學人 金融 時報 EMBA 及美國 福布斯 排名 名列 加拿大 第一名 學校 成立 1959 26 1960 第一 節課 總共 76 個學生 約克大學 已有 53 000 名 冊 學 學生 數量 加拿大 大學 第三位 學校 授予 學士 碩士 博士 學位 開設 會計 經濟 市場 營銷 國際 貿易 法律 心理 學 化學 航天 工程 音樂 戲劇 哲學 專業 課程 在校生 53 000 全職 教師 1125 兼職 教師 800 豐厚 學術 獎勵 高質量 研究 項目 加拿大 世界各地 尖學者 最終 選擇 約克大學 理由 約克大學 五個 圖書館 共計書籍 各類 雜誌 二百六十 萬冊 OsgoodeHall 法學院 圖書館 全英 聯邦 法學 圖書館 學校 生活 設施 齊全 為學生 學習 創造 環境 舒 立克 商學院'

'1069'